This is a section sequencing classifier. We used LSTM for section sequencing.

https://stackoverflow.com/questions/38189070/how-do-i-create-a-variable-length-input-lstm-in-keras

https://machinelearningmastery.com/sequence-prediction/

https://stackoverflow.com/questions/43117654/many-to-many-sequence-prediction-with-different-sequence-length


https://machinelearningmastery.com/how-to-use-an-encoder-decoder-lstm-to-echo-sequences-of-random-integers/


In [ ]:
import sklearn.preprocessing as skprep

import tensorflow as tf
from tensorflow.contrib import keras
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import datasets
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import preprocessing
from tensorflow.contrib.keras import backend as K
from tensorflow.contrib.keras import callbacks
from tensorflow.contrib.keras import utils

from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat

from itertools import repeat, chain, islice
from sklearn import preprocessing

from collections import defaultdict
import pandas as pd
import numpy as np
import os
import json
import pickle
from datetime import datetime
import sys
from sklearn import metrics,cross_validation
from tensorflow.contrib.keras.python.keras.layers.wrappers import Bidirectional

tf.logging.set_verbosity(tf.logging.WARN)


In [ ]:
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=15):
    return [randint(0, 19) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique=20):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
    # create lag copies of the sequence
    df = DataFrame(sequence)
    df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
    # drop rows with missing values
    df.dropna(inplace=True)
    # specify columns for input and output pairs
    values = df.values
    width = sequence.shape[1]
    X = values.reshape(len(values), n_in, width)
    y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
    return X, y

# prepare data for the LSTM
def get_data(n_in, n_out):
    # generate random sequence
    sequence = generate_sequence()
    # one hot encode
    encoded = one_hot_encode(sequence)
    #import pdb
    #pdb.set_trace()
    # convert to X,y pairs
    X,y = to_supervised(encoded, n_in, n_out)
    return X,y

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.80
#config.gpu_options.allow_growth=True
config.gpu_options.visible_device_list="0"
session = tf.Session(config=config)
keras.backend.set_session(session)
K.gpu_setup = ["gpu0"]

In [ ]:
# encdoing labels into numbers
class_list={}
for x in open("data/class_list.txt","r").readlines():
    x=x.strip()
    class_list[x.split(":")[0]]=x.split(":")[1].lower()

lencoder = preprocessing.LabelEncoder()
lencoder.fit_transform(class_list.values())

model_dir="lstm_sec_sequencing_evaluation1/"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

sys.stdout = open(model_dir+'report_sections_sequencing_classifier_lstm_evaluation_'+ 
                  str(datetime.now()).replace(" ","-").replace(":","-")+'.txt', 'w')



In [ ]:
# Process training and test data
filler=19
size=15
X_train={}
X_train["Sequence"]=[]
train_f=open("data/train_section_sequencing_classifier","r")

for line in train_f:
    line=list(lencoder.transform(line.strip().split(",")))
    if len(line)<1:
        continue
        
    X_train["Sequence"].append(list(islice(chain(line, repeat(filler)), size)))

#X_train = np.array(X_train["Sequence"])
X_train = X_train["Sequence"]

X_test={}
X_test["Sequence"]=[]
test_f=open("data/test_section_sequencing_classifier","r")

for line in test_f:
    line=list(lencoder.transform(line.strip().split(",")))
    if len(line)<1:
        continue

    X_test["Sequence"].append(list(islice(chain(line, repeat(filler)), size)))

#X_test= np.array(X_test["Sequence"])
X_test= X_test["Sequence"]

In [ ]:
n_epoch = 100
memory_unit=20
number_features=15  # dim of input sample. for example length of input vector.
encoded_length = 20
batch_size = 20


In [ ]:
# merge all numpy arraies of training and test
n_in = 15
n_out = 15

# for training
for index in range(len(X_train)):
    sequence = X_train[index]
    # one hot encode
    encoded = one_hot_encode(sequence)
    #import pdb
    #pdb.set_trace()
    # convert to X,y pairs
    X,y = to_supervised(encoded, n_in, n_out)
    if index ==0:
        XX_train=X
        yy_train=y
    else:
        XX_train=np.vstack((XX_train,X))
        yy_train=np.vstack((yy_train,X))
    
    
# for test
for index in range(len(X_test)):
    sequence = X_test[index]
    # one hot encode
    encoded = one_hot_encode(sequence)
    # convert to X,y pairs
    X,y = to_supervised(encoded, n_in, n_out)
    if index ==0:
        XX_test=X
        yy_test=y
    else:
        XX_test=np.vstack((XX_test,X))
        yy_test=np.vstack((yy_test,X))


In [ ]:
pickle.dump(XX_train,open(model_dir+"XX_train",'wb'))
pickle.dump(yy_train,open(model_dir+"yy_train",'wb'))
pickle.dump(XX_test,open(model_dir+"XX_test",'wb'))
pickle.dump(yy_test,open(model_dir+"yy_test",'wb'))

In [ ]:
filepath=model_dir+"weights-improvement-{epoch:02d}-{val_loss:.9f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history = callbacks.History()

tb_callback = callbacks.TensorBoard(log_dir=model_dir, histogram_freq=50, write_graph=True)


In [ ]:
# define LSTM
model = models.Sequential()
model.add(layers.LSTM(memory_unit, input_shape=(n_in, encoded_length), return_sequences=True, stateful=False))
model.add(layers.TimeDistributed(layers.Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print model.summary()

# train LSTM
# fit model for one epoch on this sequence
model.fit(XX_train, yy_train, epochs=n_epoch, batch_size=batch_size, validation_data=(XX_test, yy_test),
          verbose=1, shuffle=False,callbacks=[tb_callback, checkpoint,history])
#model.reset_states()

print "Training Done\n\n\n"

In [ ]:
# evaluate LSTM
yhat = model.predict(XX_test, batch_size=batch_size, verbose=0)
# decode all pairs
correct = 0
for i in range(len(XX_test)):
    print('Expected:', one_hot_decode(yy_test[i]), 'Predicted', one_hot_decode(yhat[i]))
    if one_hot_decode(yy_test[i]) == one_hot_decode(yhat[i]):
        correct+=1


print "correct samples: ", correct
print "Total test samples: ", len(XX_test)
print "Test Accuracy: ", correct/len(XX_test)


In [ ]:
model.save(model_dir+"final_model")

In [ ]:
print "Done!"

In [ ]:
# #load training and test data
# model_dir="lstm_sec_sequencing/"
# XX_train = pickle.load(open(model_dir+"XX_train",'rb'))
# yy_train = pickle.load(open(model_dir+"yy_train",'rb'))
# XX_test = pickle.load(open(model_dir+"XX_test",'rb'))
# yy_test = pickle.load(open(model_dir+"yy_test",'rb'))